In [39]:
import pandas as pd
import numpy as np
from sklearn import model_selection,metrics,linear_model,impute,preprocessing,ensemble

In [2]:
train = pd.read_csv('/Users/ranjithtalamanchi/Downloads/house-prices-advanced-regression-techniques/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test = pd.read_csv('/Users/ranjithtalamanchi/Downloads/house-prices-advanced-regression-techniques/test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
#Clening Data
#Finding Null Data
def findNan(df):
    nulls = df.isnull().sum()
    null_columns = [nulls.index[i] for i in range(len(nulls)) if nulls[i]>0]
    #print(df[null_columns].isnull().sum())
    return null_columns


In [5]:
train_nan_columns = findNan(train)
test_nan_columns = findNan(test)

In [6]:
def columnsG70Nan(df):
    
    nulls = df.isnull().sum()
    null_columns = [nulls.index[i] for i in range(len(nulls)) if (nulls[i] <(df.shape[0] * 0.7))]
    #print(df[null_columns].isnull().sum())
    return null_columns

In [7]:
clms_wtot70_nulls_tarin = columnsG70Nan(train)
clms_wtot70_nulls_test = columnsG70Nan(test)
#print(clms_wtot70_nulls_tarin)
#print(clms_wtot70_nulls_test)

In [8]:
# Converting Categorical data into numeric data
xTrain = train[clms_wtot70_nulls_tarin[:-1]]
yTrain = train[clms_wtot70_nulls_tarin[-1]]

xTest = test[clms_wtot70_nulls_tarin[:-1]]
#yTest = test[clms_wtot70_nulls_tarin[-1]]

'''
#xTrain.dtypes
for i in xTrain.columns:
    if xTrain[i].dtype == 'object':
        print(i,':    ',xTrain[i].unique()'
'''
encoder = preprocessing.LabelEncoder()
def convertCatToNum(df):
    
    for i in  df.columns:
        if df[i].dtype == 'object':
            df[i] = encoder.fit_transform(df[i].astype('str'))
    return df

In [9]:
xTrain = convertCatToNum(xTrain)
xTest  = convertCatToNum(xTest)
xTrain.head()

<ipython-input-8-1e4822da8945>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = encoder.fit_transform(df[i].astype('str'))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,3,65.0,8450,1,3,3,0,4,...,61,0,0,0,0,0,2,2008,8,4
1,2,20,3,80.0,9600,1,3,3,0,2,...,0,0,0,0,0,0,5,2007,8,4
2,3,60,3,68.0,11250,1,0,3,0,4,...,42,0,0,0,0,0,9,2008,8,4
3,4,70,3,60.0,9550,1,0,3,0,0,...,35,272,0,0,0,0,2,2006,8,0
4,5,60,3,84.0,14260,1,0,3,0,2,...,84,0,0,0,0,0,12,2008,8,4


In [10]:
# imputing Null values
imputer = impute.KNNImputer(n_neighbors=7)


xTrain.loc[:] = imputer.fit_transform(xTrain)
xTest.loc[:] = imputer.fit_transform(xTest)
xTrain.head()


/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,3.0,65.0,8450.0,1.0,3.0,3.0,0.0,4.0,...,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,8.0,4.0
1,2.0,20.0,3.0,80.0,9600.0,1.0,3.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,8.0,4.0
2,3.0,60.0,3.0,68.0,11250.0,1.0,0.0,3.0,0.0,4.0,...,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,8.0,4.0
3,4.0,70.0,3.0,60.0,9550.0,1.0,0.0,3.0,0.0,0.0,...,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,8.0,0.0
4,5.0,60.0,3.0,84.0,14260.0,1.0,0.0,3.0,0.0,2.0,...,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,8.0,4.0


In [11]:
#Training a model
model = linear_model.LinearRegression()
nfold = model_selection.KFold(n_splits=10,shuffle=True,random_state=42)
cross_val = model_selection.cross_val_score(model,xTrain,yTrain,cv=nfold)
print(cross_val.mean(),cross_val.max(),cross_val.min())
#model.fit(xTrain,yTrain)
#print(xTrain.shape,xTest.shape)
#pred = model.predict(xTest)

0.7925269715481518 0.9223263498148184 0.37527182549812343


In [12]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(xTrain,yTrain,test_size=0.33)
model.fit(X_train,y_train)
pred = model.predict(X_test)

In [13]:
# Calculating R2, MSE,MAE,RMSE
#print(model.coef_)
print('mean_absolute_error',metrics.mean_absolute_error(pred,y_test))
print('mean_squared_error',metrics.mean_squared_error(pred,y_test))
print('root_mean_squared_error',np.sqrt(metrics.mean_squared_error(pred,y_test)))
print('R-Square',metrics.r2_score(pred,y_test))

mean_absolute_error 21389.46498280589
mean_squared_error 2498119367.621104
root_mean_squared_error 49981.19013810199
R-Square 0.6531287229348313


In [14]:
# Ridge Regression
alpha_range=[0.68,0.69,0.7,0.71,]
ridegReg  = linear_model.RidgeCV(alphas=alpha_range,normalize=True)
ridegReg.fit(X_train,y_train)
pred = ridegReg.predict(X_test)
print('score',ridegReg.score(xTrain,yTrain))
print('mean_absolute_error',metrics.mean_absolute_error(pred,y_test))
print('mean_squared_error',metrics.mean_squared_error(pred,y_test))
print('root_mean_squared_error',np.sqrt(metrics.mean_squared_error(pred,y_test)))
print('R-Square',metrics.r2_score(pred,y_test))
print('---------------------------------------------------------')
print('alpha',ridegReg.alpha_)
#print(ridegReg.coef_)

score 0.8239737556707933
mean_absolute_error 20915.591138025837
mean_squared_error 1872964695.7634406
root_mean_squared_error 43277.76213904135
R-Square 0.6586440517645649
---------------------------------------------------------
alpha 0.68


In [15]:
#lasso Regression
lassoReg  = linear_model.LassoCV(n_alphas=1000,normalize=True)
lassoReg.fit(X_train,y_train)
pred = lassoReg.predict(X_test)
print(lassoReg.score(xTrain,yTrain))
print('mean_absolute_error',metrics.mean_absolute_error(pred,y_test))
print('mean_squared_error',metrics.mean_squared_error(pred,y_test))
print('root_mean_squared_error',np.sqrt(metrics.mean_squared_error(pred,y_test)))
print('R-Square',metrics.r2_score(pred,y_test))
print('alpha',lassoReg.alpha_)
print('---------------------------------------------------------')
#print(lassoReg.coef_)#

0.8186618133719464
mean_absolute_error 20730.443128420666
mean_squared_error 2165743587.188633
root_mean_squared_error 46537.55029208814
R-Square 0.6631555586594622
alpha 40.82990396144183
---------------------------------------------------------


In [34]:
#Elastic net Regression
elastic = linear_model.ElasticNet()
gridDict = {
    'alpha'    :[0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'l1_ratio' :np.arange(0,1,0.05)
}
nfold = model_selection.KFold(n_splits=10,shuffle=True,random_state=42)
grid = model_selection.GridSearchCV(elastic,gridDict ,scoring='neg_mean_absolute_error',cv= nfold)
print(X_train.shape,y_train.shape)
grid.fit(xTrain,yTrain)
print(grid.best_score_)
print(grid.best_params_)

(978, 76) (978,)


/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 717336420129.9756, tolerance: 787334624.1028304
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 742115540456.2096, tolerance: 830080882.4239947
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 767953412551.3986, tolerance: 839210819.7173821
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda

/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 726602153979.8925, tolerance: 814840923.2909815
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 754234081406.4967, tolerance: 843276827.3235376
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 758560095012.1521, tolerance: 846189156.0457985
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda

/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 667498526269.8961, tolerance: 846197467.0541041
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 569281553733.1598, tolerance: 833982058.1395015
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 708669844010.7909, tolerance: 814840923.2909815
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda

/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 693975522356.0706, tolerance: 830080882.4239947
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 718013150664.3799, tolerance: 839210819.7173821
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 661241439209.9492, tolerance: 802952285.151319
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3

...

/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 634216119915.0236, tolerance: 846197467.0541041
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 538207758363.3188, tolerance: 833982058.1395015
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 662084293114.2717, tolerance: 814840923.2909815
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda

/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 599058222491.6279, tolerance: 830080882.4239947
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 616927249282.8871, tolerance: 839210819.7173821
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 565183098366.3751, tolerance: 802952285.151319
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3

/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 467840704979.21234, tolerance: 843276827.3235376
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 472515625627.17834, tolerance: 846189156.0457985
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 489454711204.56445, tolerance: 842752768.8001466
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaco

-19154.1968439023
{'alpha': 0.6, 'l1_ratio': 0.45}


/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 281523927938.7046, tolerance: 842752768.8001466
  model = cd_fast.enet_coordinate_descent(
/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 781431487784.7762, tolerance: 920791133.4609975
  model = cd_fast.enet_coordinate_descent(


In [38]:
print(grid.best_score_)
print(grid.best_params_)
#linear_model.ElasticNet?
elastic1 = linear_model.ElasticNet(alpha=0.69,l1_ratio=0.5)
elastic1.fit(xTrain,yTrain)
print(elastic1.score(xTrain,yTrain))
pred = elastic1.predict(xTest)

-19154.1968439023
{'alpha': 0.6, 'l1_ratio': 0.45}
0.8405836924833385


/Users/ranjithtalamanchi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 779874929583.7904, tolerance: 920791133.4609975
  model = cd_fast.enet_coordinate_descent(


In [32]:
sdf = pd.DataFrame({'ID':test.Id,'SalePrice':pred})
sdf.to_csv('/Users/ranjithtalamanchi/Downloads/house-prices-advanced-regression-techniques/submission.csv',index=False)

In [65]:
X_train, X_test, X_train, y_test = model_selection.train_test_split(xTrain,yTrain,test_size=0.33)
regTree = ensemble.RandomForestRegressor(n_estimators=100,random_state=42)
regTree.fit(xTrain,yTrain)
print(regTree.score(xTrain,yTrain))
pred = regTree.predict(xTest)
sub = pd.read_csv('/Users/ranjithtalamanchi/Downloads/house-prices-advanced-regression-techniques/sample_submission.csv')
print((metrics.mean_squared_log_error(pred,sub.SalePrice)))

0.9808665036818569
0.12710995165570693


In [66]:
sdf = pd.DataFrame({'ID':test.Id,'SalePrice':pred})
sdf.to_csv('/Users/ranjithtalamanchi/Downloads/house-prices-advanced-regression-techniques/submission.csv',index=False)

In [59]:
sub = pd.read_csv('/Users/ranjithtalamanchi/Downloads/house-prices-advanced-regression-techniques/sample_submission.csv')

In [60]:
sub.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
